# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [6]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [13]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [14]:
from pathlib import Path
import os
current_directory = os.getcwd()
results_folder = Path(current_directory + '/fenicsx/hyperelasticity/results/')
results_folder.mkdir(exist_ok=True, parents=True)

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif(str(results_folder) + "/displacement.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

Finally, we solve the problem over several time steps, updating the z-component of the traction

In [15]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-09-17 19:08:05.652 (   4.040s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:07.412 (   5.801s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:08.929 (   7.318s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2024-09-17 19:08:09.092 (   7.481s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:10.613 (   9.002s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2024-09-17 19:08:10.775 (   9.163s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:12.268 (  10.656s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2024-09-17 19:08:19.377 (  17.766s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:20.991 (  19.380s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:22.453 (  20.842s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2024-09-17 19:08:22.616 (  21.005s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:24.082 (  22.471s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2024-09-17 19:08:24.245 (  22.634s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:25.820 (  24.209s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2024-09-17 19:08:32.883 (  31.272s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:34.363 (  32.751s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 9: r (abs) = 1.70763e-10 (tol = 1e-08) r (rel) = 1.16148e-12(tol = 1e-08)
2024-09-17 19:08:34.363 (  32.752s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 9 iterations and 9 linear solver iterations.
2024-09-17 19:08:34.561 (  32.949s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:36.240 (  34.629s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:08:37.743 (  36.131s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2024-09-17 19:08:37.904 (  36.293s) [main            ]              petsc.c

...

2024-09-17 19:09:41.510 (  99.899s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 6: r (abs) = 1.78865e-08 (tol = 1e-08) r (rel) = 4.34708e-10(tol = 1e-08)
2024-09-17 19:09:41.510 (  99.899s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.
2024-09-17 19:09:41.708 ( 100.097s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:43.352 ( 101.741s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:44.811 ( 103.200s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2024-09-17 19:09:44.972 ( 103.360s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:46.449 ( 104.837s) [main            ]       NewtonSolver.cp

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2024-09-17 19:09:51.589 ( 109.978s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:53.229 ( 111.618s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:54.835 ( 113.224s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2024-09-17 19:09:55.000 ( 113.389s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:56.556 ( 114.944s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2024-09-17 19:09:56.721 ( 115.110s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 19:09:58.239 ( 116.628s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteratio

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


2024-09-17 19:10:01.491 ( 119.880s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 6: r (abs) = 2.28231e-13 (tol = 1e-08) r (rel) = 7.86338e-15(tol = 1e-08)
2024-09-17 19:10:01.491 ( 119.880s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.


![Deformation](results/displacement.gif)
